# # Picking the “Right” Number of Geo‐Clusters
We'll evaluate both **inertia** (elbow method) and **silhouette score** for pickup and dropoff coordinates.

# 1) Imports

In [1]:
import warnings

import folium
import hdbscan
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.collections import mcolors
from shapely.geometry.multipoint import MultiPoint
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors

from constants.taxi_c import TAXI_PROCESSED_CSV, GeoBounds

warnings.filterwarnings(
    "ignore",
    message=".*'force_all_finite' was renamed to 'ensure_all_finite'.*",
    category=FutureWarning,
    module="sklearn"
)

# 2) Load your cleaned+feature‐engineered taxi DataFrame

In [2]:
df = pd.read_csv(TAXI_PROCESSED_CSV)

In [3]:
# Create a mask for rows within GeoBounds for both pickup and drop‐off coordinates
mask = (
    (df["pickup_latitude"] > GeoBounds.min_lat) &
    (df["pickup_latitude"] < GeoBounds.max_lat) &
    (df["pickup_longitude"] > GeoBounds.min_lon) &
    (df["pickup_longitude"] < GeoBounds.max_lon) &
    (df["dropoff_latitude"] > GeoBounds.min_lat) &
    (df["dropoff_latitude"] < GeoBounds.max_lat) &
    (df["dropoff_longitude"] > GeoBounds.min_lon) &
    (df["dropoff_longitude"] < GeoBounds.max_lon)
)

In [4]:
def run_hdbscan_sample(coords_deg, *,
    min_cluster_size=150,
    min_samples=None,
    sample_size=100_000,
    seed=42):
  n = len(coords_deg)
  rng = np.random.default_rng(seed)
  samp_ix = rng.choice(n, min(n, sample_size), replace=False)
  samp = coords_deg[samp_ix]
  coords_rad = np.radians(samp)

  clusterer = hdbscan.HDBSCAN(
      min_cluster_size=min_cluster_size,
      min_samples=min_samples or min_cluster_size // 10,
      metric="haversine",
      cluster_selection_method="leaf",
      prediction_data=True,
      core_dist_n_jobs=-1,
  )
  clusterer.fit(coords_rad)

  full_lbl = np.full(n, -1, dtype=int)
  full_lbl[samp_ix] = clusterer.labels_

  core_mask = clusterer.labels_ >= 0
  if core_mask.any():
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(coords_rad[core_mask])
    full_coords_rad = np.radians(coords_deg)
    dist, idx = nbrs.kneighbors(full_coords_rad, return_distance=True)
    nn_lbl = clusterer.labels_[core_mask][idx.ravel()]
    try:
      threshold = clusterer.minimum_spanning_tree_.max()
    except AttributeError:
      threshold = np.inf
    mask = (full_lbl == -1) & (dist.ravel() <= threshold)
    full_lbl[mask] = nn_lbl[mask]

  return full_lbl

In [5]:
def optimize_cluster_params(df, cluster_type='pickup', param_grid=None, sample_size=10_000,
    seed=42,
    min_silhouette_threshold=0.2):
  """
  Optimizes clustering parameters using silhouette score.
  Applies a minimum silhouette score threshold to filter out poor results.

  Args:
      df (pd.DataFrame): DataFrame with coordinates.
      cluster_type (str): 'pickup' or 'dropoff'.
      param_grid (list): List of dicts with parameters.
      sample_size (int): Sample size for evaluation.
      seed (int): Random seed.
      min_silhouette_threshold (float): Minimum acceptable silhouette score.

  Returns:
      dict: Best parameter combination and a DataFrame summary.
  """
  if param_grid is None:
    # Expanded grid for NYC cab pickup coordinates
    param_grid = [{
      'min_cluster_size': size,
      'min_samples': max(2, size // 10)
    } for size in
      [620, 630, 670, 680, 720, 730, 770, 780, 810, 820, 830, 840
       ]]

  coord_cols = ['pickup_latitude', 'pickup_longitude'] if cluster_type == 'pickup' else [
    'dropoff_latitude', 'dropoff_longitude']
  df_sample = df.sample(n=sample_size, random_state=seed)
  coords = df_sample[coord_cols].to_numpy()

  results = []
  best_score = -1
  best_params = None

  for params in param_grid:
    labels = run_hdbscan_sample(coords,
                                min_cluster_size=params['min_cluster_size'],
                                min_samples=params['min_samples'],
                                sample_size=sample_size,
                                seed=seed)
    mask = labels != -1
    unique_labels = np.unique(labels[mask])
    if len(unique_labels) <= 1:
      score = -1
    else:
      try:
        score = silhouette_score(coords[mask], labels[mask], random_state=42)
        if score < min_silhouette_threshold:
          score = -1
      except Exception:
        score = -1
    results.append({
      'min_cluster_size': params['min_cluster_size'],
      'min_samples': params['min_samples'],
      'silhouette_score': score,
      'n_clusters': len(unique_labels)
    })
    if score > best_score:
      best_score = score
      best_params = params.copy()

  results_df = pd.DataFrame(results)
  return {'best_params': best_params, 'results': results_df}

In [6]:
# print(optimize_cluster_params(df, cluster_type='pickup'))

{'best_params': {'min_cluster_size': 40, 'min_samples': 4}, 'results':    min_cluster_size  min_samples  silhouette_score  n_clusters
0                40            4          0.235593          87
1                45            4          0.233076          72
2                50            5          0.217372          67

In [7]:
# print(optimize_cluster_params(df, cluster_type='dropoff'))

In [8]:
coords = df.loc[mask, ["pickup_latitude", "pickup_longitude"]].to_numpy()
pickup_cluster_labels = run_hdbscan_sample(coords,
                                           min_cluster_size=730,
                                           min_samples=73)
df["pickup_cluster_hdb"] = -1
df.loc[mask, "pickup_cluster_hdb"] = pickup_cluster_labels

In [12]:
coords = df.loc[mask, ["dropoff_latitude", "dropoff_longitude"]].to_numpy()
dropoff_cluster_labels = run_hdbscan_sample(coords,
                                            min_cluster_size=630,
                                            min_samples=63)
df["dropoff_cluster_hdb"] = -1
df.loc[mask, "dropoff_cluster_hdb"] = dropoff_cluster_labels

In [11]:
df_z = df.sample(n=50000, random_state=42)

# — base map centered on NYC —
m = folium.Map(
    location=[40.75, -73.97],
    zoom_start=11,
    tiles="CartoDB positron"
)

# — prepare a color palette —
cluster_ids = sorted(df_z["pickup_cluster_hdb"].unique())
cmap = plt.colormaps["Set2"].resampled(len(cluster_ids))

palette = [mcolors.to_hex(cmap(i)) for i in range(len(cluster_ids))]
color_map = {cid: palette[i] for i, cid in enumerate(cluster_ids)}

# — for each cluster, compute its convex hull and draw it —
for cid in cluster_ids:
  pts = df_z[df_z["pickup_cluster_hdb"] == cid][["pickup_longitude", "pickup_latitude"]].values
  if len(pts) < 3:
    continue  # need at least 3 points for a polygon
hull = MultiPoint(pts).convex_hull
# extract hull coords and swap to (lat, lon)
hull_coords = [(lat, lon) for lon, lat in hull.exterior.coords]
folium.Polygon(
    locations=hull_coords,
    color=color_map[cid],
    weight=2,
    fill=True,
    fill_color=color_map[cid],
    fill_opacity=0.2,
    popup=f"Cluster {cid}"
).add_to(m)

# — also plot the centroids —
centroids = df.groupby("pickup_cluster_hdb")[["pickup_latitude", "pickup_longitude"]].mean()
for cid, row in centroids.iterrows():
  folium.CircleMarker(
      location=(row["pickup_latitude"], row["pickup_longitude"]),
      radius=6,
      color="black",
      fill=True,
      fill_color=color_map[cid],
      fill_opacity=1.0,
      popup=f"Centroid {cid}"
  ).add_to(m)

m

In [14]:
m.save("../figures/pickup_clusters_hdb.html")
# --- save static PNG image ---
png = m._to_png(5)  # 5× pixel ratio for decent resolution
with open("../figures/pickup_clusters_hdb.png", "wb") as f:
  f.write(png)

In [13]:
df_z = df.sample(n=50000, random_state=42)

# — base map centered on NYC —
m = folium.Map(
    location=[40.75, -73.97],
    zoom_start=11,
    tiles="CartoDB positron"
)

# — prepare a color palette —
cluster_ids = sorted(df_z["dropoff_cluster_hdb"].unique())
cmap = plt.colormaps["Set2"].resampled(len(cluster_ids))

palette = [mcolors.to_hex(cmap(i)) for i in range(len(cluster_ids))]
color_map = {cid: palette[i] for i, cid in enumerate(cluster_ids)}

# — for each cluster, compute its convex hull and draw it —
for cid in cluster_ids:
  pts = df_z[df_z["dropoff_cluster_hdb"] == cid][["dropoff_longitude", "dropoff_latitude"]].values
  if len(pts) < 3:
    continue  # need at least 3 points for a polygon
  hull = MultiPoint(pts).convex_hull
  # extract hull coords and swap to (lat, lon)
  hull_coords = [(lat, lon) for lon, lat in hull.exterior.coords]
  folium.Polygon(
      locations=hull_coords,
      color=color_map[cid],
      weight=2,
      fill=True,
      fill_color=color_map[cid],
      fill_opacity=0.2,
      popup=f"Cluster {cid}"
  ).add_to(m)

# — also plot the centroids —
centroids = df.groupby("dropoff_cluster_hdb")[["dropoff_latitude", "dropoff_longitude"]].mean()
for cid, row in centroids.iterrows():
  folium.CircleMarker(
      location=(row["dropoff_latitude"], row["dropoff_longitude"]),
      radius=6,
      color="black",
      fill=True,
      fill_color=color_map[cid],
      fill_opacity=1.0,
      popup=f"Centroid {cid}"
  ).add_to(m)

m

In [15]:
m.save("../figures/dropoff_clusters_hdb.html")
# --- save static PNG image ---
png = m._to_png(5)  # 5× pixel ratio for decent resolution
with open("../figures/dropoff_clusters_hdb.png", "wb") as f:
  f.write(png)

In [17]:
from pathlib import Path

if True:
  Path(TAXI_PROCESSED_CSV).parent.mkdir(parents=True, exist_ok=True)
  df.to_csv(TAXI_PROCESSED_CSV, index=False)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 40 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1458644 non-null  object 
 1   vendor_id                  1458644 non-null  int64  
 2   pickup_datetime            1458644 non-null  object 
 3   dropoff_datetime           1458644 non-null  object 
 4   passenger_count            1458644 non-null  int64  
 5   pickup_longitude           1458644 non-null  float64
 6   pickup_latitude            1458644 non-null  float64
 7   dropoff_longitude          1458644 non-null  float64
 8   dropoff_latitude           1458644 non-null  float64
 9   store_and_fwd_flag         1458644 non-null  object 
 10  trip_duration              1458644 non-null  int64  
 11  is_group_trip              1458644 non-null  int64  
 12  passenger_count_invalid    1458644 non-null  int64  
 13  pickup_longi